In [1]:
#Author: Anshul Kapoor
#Code Purpose: Make Single JSON for all EGamma SFs

#Names that you want for SFs for files above (same order please)
namesSFs=["Loose", "Medium", "Tight", "wp80", "wp90"]

#All the files for SFs
files2016preVFP={
    "Loose":"EGdata/SF-Repository/UL16/preVFP/Photons/Loose/egammaEffi.txt_EGM2D.root",
    "Medium":"EGdata/SF-Repository/UL16/preVFP/Photons/Medium/egammaEffi.txt_EGM2D.root",
    "Tight":"EGdata/SF-Repository/UL16/preVFP/Photons/Tight/egammaEffi.txt_EGM2D.root",
    "wp80":"EGdata/SF-Repository/UL16/preVFP/Photons/MVA80/egammaEffi.txt_EGM2D.root",
    "wp90":"EGdata/SF-Repository/UL16/preVFP/Photons/MVA90/egammaEffi.txt_EGM2D.root",
}


files2016postVFP={
    "Loose":"EGdata/SF-Repository/UL16/postVFP/Photons/Loose/egammaEffi.txt_EGM2D_Pho_Loose_UL16_postVFP.root",
    "Medium":"EGdata/SF-Repository/UL16/postVFP/Photons/Medium/egammaEffi.txt_EGM2D.root",
    "Tight":"EGdata/SF-Repository/UL16/postVFP/Photons/Tight/egammaEffi.txt_EGM2D_Pho_Tight_UL16_postVFP.root",
    "wp80":"EGdata/SF-Repository/UL16/postVFP/Photons/MVA80/egammaEffi.txt_EGM2D_Pho_MVA80_UL16_postVFP.root",
    "wp90":"EGdata/SF-Repository/UL16/postVFP/Photons/MVA90/egammaEffi.txt_EGM2D_Pho_MVA90_UL16_postVFP.root",
}

files2017={
    "Loose":"EGdata/SF-Repository/UL17/Photons/Loose/passingLoose100XV2_lowpTChebychev_addGaus/egammaEffi.txt_EGM2D.root",
    "Medium":"EGdata/SF-Repository/UL17/Photons/Medium/egammaEffi.txt_EGM2D.root",
    "Tight":"EGdata/SF-Repository/UL17/Photons/Tight/egammaEffi.txt_EGM2D.root",
    "wp80":"EGdata/SF-Repository/UL17/Photons/MVA80/passingMVA94XV2wp80/egammaEffi.txt_EGM2D.root",
    "wp90":"EGdata/SF-Repository/UL17/Photons/MVA90/passingMVA94XV2wp90/egammaEffi.txt_EGM2D.root",
}

files2018={
    "Loose":"EGdata/SF-Repository/UL18/Photons/Loose/passingLoose100XV2/egammaEffi.txt_EGM2D.root",
    "Medium":"EGdata/SF-Repository/UL18/Photons/Medium/passingMedium100XV2/egammaEffi.txt_EGM2D.root",
    "Tight":"EGdata/SF-Repository/UL18/Photons/Tight/passingTight100XV2/egammaEffi.txt_EGM2D.root",
    "wp80":"EGdata/SF-Repository/UL18/Photons/MVA80/passingMVA94XV2wp80/egammaEffi.txt_EGM2D.root",
    "wp90":"EGdata/SF-Repository/UL18/Photons/MVA90/passingMVA94XV2wp90/egammaEffi.txt_EGM2D.root",
}





#Names that you want for errors for files above (same order please)
nameJSON="egcorrs_Photons_UL.json" # Name of final JSON

sfhist="EGamma_SF2D"

In [2]:
# Technical part below
from ROOT import TFile, TH1F, TCanvas, TString
import correctionlib.schemav2 as schema
from correctionlib.schemav2 import Correction
corrs=[]
#Function to extract SFs from EGamma standard root files
def getSFs(fn="filename",IsSF="sf"):
    tf = TFile(fn)
    fo = tf.Get(sfhist)
    Xbins=fo.GetNbinsX()
    Ybins=fo.GetNbinsY()
    X=[None]*(Xbins+1)
    Y=[None]*(Ybins+1)
    values=[]
    errors=[]
    for i in range(1,Xbins + 1):
        X[i-1]=(fo.GetXaxis().GetBinLowEdge(i))
    X[Xbins]=fo.GetXaxis().GetBinUpEdge(Xbins)
    for j in range(1,Ybins + 1):
        Y[j-1]=(fo.GetYaxis().GetBinLowEdge(j))
    Y[Ybins]=fo.GetYaxis().GetBinUpEdge(Ybins)
    for i in range(1,Xbins + 1):
        for j in range(1,Ybins + 1):
            values.append(fo.GetBinContent(i,j))
            errors.append(fo.GetBinError(i,j))
    if IsSF=="sf":
        valSFs=schema.MultiBinning.parse_obj({
            "inputs":["eta","pt"],
            "nodetype": "multibinning",
            "edges": [
                X,
                Y,
            ],
            "content": values,
            "flow": 'error',
        })
        return valSFs
    if IsSF=="syst":
        valerrors=schema.MultiBinning.parse_obj({
            "inputs":["eta","pt"],
            "nodetype": "multibinning",
            "edges": [
                X,
                Y,
            ],
            "content": errors,
            "flow": 'error',
        })
        return valerrors

Welcome to JupyROOT 6.22/06


In [3]:
def SFyearwise(files=[],names=[],valtypes=["sf","syst"]):
    output = schema.Category.parse_obj({
                "nodetype": "category",
                "input": "ValType",
                "content":[
                    schema.CategoryItem.parse_obj({
                        "key": val, 
                        "value": schema.Category.parse_obj({
                            "nodetype": "category",
                            "input": "WorkingPoint",
                            "content":[
                                schema.CategoryItem.parse_obj({
                                        "key": name, 
                                        "value": getSFs(fn=files[name],IsSF=val)})
                                for name in names
                                ],
                            })
                    })
                    for val in valtypes
                ],
    })
    return output    

In [4]:
corr = Correction.parse_obj(
    {
        "version": 2,
        "name": "PhotonsUL",
        "inputs": [
            {"name": "year","type": "string"},
            {"name": "ValType","type": "string"},
            {"name": "WorkingPoint","type": "string"},
            {"type": "real", "name": "eta", "description": "possibly supercluster eta?"},
            {"name": "pt", "type": "real"},
        ],
        "output": {"name": "weight", "type": "real"},
        "data": schema.Category.parse_obj({
            "nodetype": "category",
            "input": "year",
            "content": [
                schema.CategoryItem.parse_obj({"key":"2017",
                                               "value": SFyearwise(files=files2017,names=namesSFs)}),
                schema.CategoryItem.parse_obj({"key":"2018",
                                               "value": SFyearwise(files=files2018,names=namesSFs)}),
                schema.CategoryItem.parse_obj({"key":"2016preVFP",
                                               "value": SFyearwise(files=files2016preVFP,names=namesSFs)}),
                schema.CategoryItem.parse_obj({"key":"2016postVFP",
                                               "value": SFyearwise(files=files2016postVFP,names=namesSFs)}),]
        })
    })
        
corrs.append(corr)

#Save JSON
from correctionlib.schemav2 import CorrectionSet
import gzip

cset = CorrectionSet(schema_version=2, corrections=corrs)

with open(nameJSON, "w") as fout:
    fout.write(cset.json(exclude_unset=True, indent=4))

In [5]:
#Evaluator Example

#import libcorrection

#cset = libcorrection.CorrectionSet(nameJSON)

from correctionlib import _core
evaluator = _core.CorrectionSet.from_string(cset.json(exclude_unset=True, indent=4))


In [6]:
valsf= evaluator["PhotonsUL"].evaluate("2017","sf","Medium",1.1, 34.0)
print("sf is:"+str(valsf))

valsyst= evaluator["PhotonsUL"].evaluate("2016postVFP","syst","Medium",1.1, 34.0)
print("syst is:"+str(valsyst))

sf is:0.9985875487327576
syst is:0.023524939995167835
